In [ ]:
!pip install git+https://github.com/onnx/onnx-tensorflow.git
!pip install tensorflowjs

In [1]:
import io
import os, sys
import requests
import PIL

import torch
import torchvision.transforms as T
# import torchvision.transforms.functional as TF

from IPython.display import display, display_markdown



def download_image(url):
    resp = requests.get(url)
    resp.raise_for_status()
    return PIL.Image.open(io.BytesIO(resp.content))

# target_image_size = 1024
# def preprocess(img):
#     s = min(img.size)
    
#     if s < target_image_size:
#         raise ValueError(f'min dim for image {s} < {target_image_size}')
        
#     r = target_image_size / s
#     s = (round(r * img.size[1]), round(r * img.size[0]))
#     img = TF.resize(img, s, interpolation=PIL.Image.LANCZOS)
#     img = TF.center_crop(img, output_size=2 * [target_image_size])
#     img = torch.unsqueeze(T.ToTensor()(img), 0)
#     mapped = map_pixels(img)
#     return mapped

In [2]:
# img = download_image('https://i.imgur.com/ayV3L3O.jpg') #1024x1024
img = download_image('https://i.imgur.com/hFUANgu.jpg') #512x512

In [3]:
import os
import collections
from typing import Union, List
import numpy as np
from PIL import Image
import PIL.Image
import PIL.ImageFile
import numpy as np
import scipy.ndimage

import os
os.system("git clone https://github.com/bryandlee/animegan2-pytorch")
os.system("gdown https://drive.google.com/uc?id=1WK5Mdt6mwlcsqCZMHkCUSDJxN1UyFi0-")
os.system("gdown https://drive.google.com/uc?id=18H3iK09_d54qEDoWIc82SyWB2xun4gjU")
import sys
sys.path.append("animegan2-pytorch")
import torch
# torch.set_grad_enabled(False)
from model import Generator
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Generator().eval().to(device)
model.load_state_dict(torch.load("face_paint_512_v2_0.pt"))
from PIL import Image
from torchvision.transforms.functional import to_tensor, to_pil_image

input = to_tensor(img).unsqueeze(0) * 2 - 1

In [10]:
print(input)
output = model(input.to(device)).cpu()[0]
print(output)
output = (output * 0.5 + 0.5).clip(0, 1)
print(output)
out_img = to_pil_image(output)

tensor([[[[-0.6392, -0.6392, -0.6392,  ..., -0.3647, -0.3647, -0.3647],
          [-0.6392, -0.6392, -0.6392,  ..., -0.3647, -0.3647, -0.3647],
          [-0.6392, -0.6392, -0.6392,  ..., -0.3647, -0.3647, -0.3647],
          ...,
          [ 0.4510,  0.4510,  0.4510,  ...,  0.5373,  0.5451,  0.5608],
          [ 0.5059,  0.5059,  0.5059,  ...,  0.5373,  0.5451,  0.5608],
          [ 0.5373,  0.5373,  0.5373,  ...,  0.5373,  0.5451,  0.5608]],

         [[-0.0275, -0.0275, -0.0275,  ...,  0.1137,  0.1137,  0.1137],
          [-0.0275, -0.0275, -0.0275,  ...,  0.1137,  0.1137,  0.1137],
          [-0.0275, -0.0275, -0.0275,  ...,  0.1137,  0.1137,  0.1137],
          ...,
          [ 0.5294,  0.5294,  0.5294,  ...,  0.5608,  0.5686,  0.5843],
          [ 0.5843,  0.5843,  0.5843,  ...,  0.5608,  0.5686,  0.5843],
          [ 0.6157,  0.6157,  0.6157,  ...,  0.5608,  0.5686,  0.5843]],

         [[ 0.5529,  0.5529,  0.5529,  ...,  0.6392,  0.6392,  0.6392],
          [ 0.5529,  0.5529,  

In [ ]:
display(out_img)

In [11]:
torch.onnx.export(model, input.to(device), "anime-gan-v2.onnx", export_params=True, opset_version=12) 

In [ ]:
!onnx-tf convert -i anime-gan-v2.onnx -o anime-gan-v2-tf

In [ ]:
!tensorflowjs_converter --input_format tf_saved_model ./anime-gan-v2-tf ./anime-gan-v2-tfjs

In [2]:
import tensorflow as tf

# Convert the model to tflite
converter = tf.lite.TFLiteConverter.from_saved_model("./anime-gan-v2-tf") # path to the SavedModel directory
tflite_model = converter.convert()

# Save the model.
with open('anime-gan-v2.tflite', 'wb') as f:
  f.write(tflite_model)